In [12]:
from model_selection import load_dataset, df_train_test_split
from settings import BASE_DIR
import Preprocess as ps
from nltk.stem import SnowballStemmer
import TFIDF_Models as cnb
import scipy.stats.distributions as dists
from sklearn.metrics import accuracy_score

# explicit negations
negations = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
            "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
            "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
            "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
            "mustn't":"must not"}

# convert twitter emojis in twitch style emojis
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad',
      ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
      ':-@': 'shocked', ':@': 'shocked',':-$': 'confused',
      ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
      '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
      '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink',
      ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

regex_subs = {
                r"https?://[^s]+" : "URL", # replace any url with URL
                "www.[^ ]+" : "URL", # replace any url with URL
                r"@[^\s]+" : "USR", # replace any user tag with USR (the tag system is the same also in twitch)
                r"(.)\1\1+" : r"\1\1", # replace 3 consecutive chars with 2
                r"[\s]+" : " ", # remove consec spaces
                "#[a-z0-9]*" : "" #remove hashtags, they are not used in twitch chats
             }

sbStem = SnowballStemmer("english", True)

In [3]:
path = "{BaseDir}/Advanced_Machine_Learning_Project/data/dataset.csv".format(BaseDir=BASE_DIR)
columns = ["sentiment", "ids", "date", "flag", "user", "text"]
final_columns = ["text", "sentiment"]
df = load_dataset(path, columns, final_columns)

loading dataset...
...dataset loaded


In [4]:
preprocess = ps.Preprocess(negations, emojis, regex_subs, sbStem)
df = preprocess.df_pre_process(df, "text", "sentiment")
X_train, X_test, Y_train, Y_test = df_train_test_split(df, "text", "sentiment",test_size=0.05)

starting preprocessing...
...preprocessing completed
starting splitting dataset...
...dataset splitted


In [5]:
a = RandomizedSearchCV(estimator=Pipeline(steps=[('model',
                                              W2VDecisionTreeClassifier())]),
                   n_iter=1, n_jobs=1,
                   param_distributions=[{'model': [W2VDecisionTreeClassifier()],
                                         'model__ccp_alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f28494cdc70>,
                                         'model__criterion': ['gini',
                                                              'entropy'],
                                         'model__epochs': <scipy.stats._distn_infrastructure.rv_frozen ob...
                                         'model__max_iter': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2849666580>,
                                         'model__ngram_range': [(1, 2)],
                                         'model__penalty': ['l1', 'l2',
                                                            'elasticnet',
                                                            'none'],
                                         'model__solver': ['saga'],
                                         'model__tfidf_max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2849a06e80>,
                                         'model__tol': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2849a06fa0>}],
                   return_train_score=True)
params_grid =[dict(model = [cnb.TFIDFLogisticRegression()],
              model__tfidf_max_features=dists.randint(1, 50000000),
              model__ngram_range=[(1, 2)],
              model__penalty=['l1', 'l2', 'elasticnet', 'none'],
              model__tol=dists.uniform(0.00001, 0.001),
              model__C=dists.uniform(0.1, 2.0),
              model__fit_intercept=[True, False],
              model__solver=['saga'],
              model__max_iter=dists.randint(100, 1000+1),
              model__l1_ratio = dists.uniform(0.0,1.0))]
#
# apply_random_search(pip, params_grid, X_train, Y_train)

LogReg = cnb.TFIDFLogisticRegression()
model = LogReg.fit(X_train, Y_train)

# model = search['params'][0]
# model['model']=str(model['model'])
# model['model'] = model['model'][:model['model'].find('(')]
# df = {'model' : [model['model']],
#       'mean_fit_time' : [search['mean_fit_time'][0]],
#       'mean_score_time': [search['mean_score_time'][0]],
#       'mean_train_score' : [search['mean_train_score'][0]],
#       'mean_test_score' : [search['mean_test_score'][0]]}
# results = results.append(pd.DataFrame(df))
# print(results)



starting vectorizing words...
...words vectorized


In [7]:
prediction = model.predict(X_test)

starting vectorizing words...
...words vectorized


In [13]:
accuracy_score(Y_test, prediction)

0.6760375

In [14]:
df

,text,sentiment
0,usr url a bummer. you shoulda got david carr o...,0
1,is upset that he can not updat his facebook by...,0
2,usr i dive mani time for the ball. manag to sa...,0
3,my whole bodi feel itchi and like its on fire,0
4,"usr no, it's not behav at all. i'm mad. why am...",0
...,...,...
1599995,just woke up. having no school is the best fee...,1
1599996,thewdb.com - very cool to hear old walt interv...,1
1599997,are you readi for your mojo makeover? ask me f...,1
1599998,happi 38th birthday to my boo of all time!! tu...,1


In [15]:
X_test

980580                     had garlic bread after long time 
718900            at a famili grad party. so. many. drinks. 
1161001    do not forget to watch the mtv movi awards! ne...
1116629    usr hahaha no its not a bad thing! i just had ...
656884     went to the dentist over 2 hour ago and i can ...
                                 ...                        
1323144    usr we do not have a booth presence, but some ...
413519                usr ..and i only just got the old one 
1023844    : think bout the lost -- and the cross &amp; p...
1081151    usr the door is wide open for you!! all day ev...
274864                           usr i think about it too.. 
Name: text, Length: 80000, dtype: object

In [16]:
prediction

array([0, 0, 1, ..., 1, 1, 0])